In [5]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
!pip install -q keras-cv

In [7]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras_cv
import keras

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!pip install kaggle


In [ ]:
!kaggle competitions list


ref                                                                                 deadline             category                reward  teamCount  userHasEntered  
----------------------------------------------------------------------------------  -------------------  ---------------  -------------  ---------  --------------  
https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-3       2026-04-15 23:59:00  Featured         2,207,152 Usd       1871           False  
https://www.kaggle.com/competitions/vesuvius-challenge-surface-detection            2026-02-13 23:59:00  Research           200,000 Usd       1233           False  
https://www.kaggle.com/competitions/stanford-rna-3d-folding-2                       2026-03-25 23:59:00  Featured           100,000 Usd        743           False  
https://www.kaggle.com/competitions/med-gemma-impact-challenge                      2026-02-24 23:59:00  Featured           100,000 Usd        104           False  
https://ww

In [9]:
!kaggle competitions download -c terra-seg-rugged-terrain-segmentation --force

100% 3.59G/3.60G [01:01<00:00, 41.0MB/s]
100% 3.60G/3.60G [01:01<00:00, 63.3MB/s]


In [ ]:
!ls

kaggle.json  sample_data  terra-seg-rugged-terrain-segmentation.zip


In [10]:
!unzip -q terra-seg-rugged-terrain-segmentation.zip


In [12]:
!pip install -q tensorflow pandas pillow opencv-python tqdm scikit-learn

In [13]:
# All imports and configuration
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import ResNet50
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd
import albumentations as A

#================= CONFIGURATION =================
# Dataset paths
TRAIN_IMAGES_DIR = "/content/offroad-seg-kaggle/train_images"
TRAIN_MASKS_DIR = "/content/offroad-seg-kaggle/train_masks"
TEST_IMAGES_DIR = "/content/offroad-seg-kaggle/test_images_padded"

# Google Drive paths
CHECKPOINT_DIR = "/content/drive/MyDrive/TerraSeg_Hackathon/checkpoints"
SUBMISSION_DIR = "/content/drive/MyDrive/TerraSeg_Hackathon/submissions"

# Training settings
IMG_SIZE = (512, 512)
BATCH_SIZE = 4
EPOCHS = 20
NUM_CLASSES = 10  # All 10 classes
LEARNING_RATE = 1e-4

# ALL 10 mask values
MASK_VALUES = [100, 200, 300, 500, 550, 600, 700, 800, 7100, 10000]
VALUE_TO_CLASS = {v: i for i, v in enumerate(MASK_VALUES)}
CLASS_TO_VALUE = {i: v for i, v in enumerate(MASK_VALUES)}

# Foreground classes for binary mask
FG_VALUES = {7100, 10000}

# TTA
USE_TTA = True
#=================================================

os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(SUBMISSION_DIR, exist_ok=True)
print("GPUs:", tf.config.list_physical_devices('GPU'))
print(f"Classes: {NUM_CLASSES}, Image size: {IMG_SIZE}")

GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Classes: 10, Image size: (512, 512)


In [14]:
def convolution_block(block_input, num_filters=256, kernel_size=3, dilation_rate=1, use_bias=False):
    x = layers.Conv2D(num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate,
                      padding="same", use_bias=use_bias, kernel_initializer='he_normal')(block_input)
    x = layers.BatchNormalization()(x)
    return layers.ReLU()(x)

def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]),
                                    interpolation="bilinear")(x)
    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)
    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    return convolution_block(x, kernel_size=1)

def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(image_size[0], image_size[1], 3))
    resnet50 = ResNet50(weights="imagenet", include_top=False, input_tensor=model_input)

    low_level = resnet50.get_layer("conv2_block3_out").output
    high_level = resnet50.get_layer("conv4_block6_out").output
    x = DilatedSpatialPyramidPooling(high_level)

    x = layers.UpSampling2D(size=(image_size[0]//4//x.shape[1], image_size[1]//4//x.shape[2]),
                            interpolation="bilinear")(x)
    low_level = convolution_block(low_level, num_filters=48, kernel_size=1)
    x = layers.Concatenate(axis=-1)([x, low_level])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(size=(image_size[0]//x.shape[1], image_size[1]//x.shape[2]),
                            interpolation="bilinear")(x)
    return Model(inputs=model_input, outputs=layers.Conv2D(num_classes, 1, padding="same")(x))

model = DeeplabV3Plus(IMG_SIZE, NUM_CLASSES)
print(f"Model: {NUM_CLASSES} classes, {model.count_params():,} params")

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Model: 10 classes, 17,832,778 params


In [15]:
# OPTIMIZED augmentation based on dataset analysis:
# - Brightness varies 28-187 (136%) -> strong brightness/contrast OK
# - Colors vary by 150+ per channel -> RGB shift OK
# - Horizon varies by 37% of image -> small rotation OK
# - NO vertical flip (sky=top, ground=bottom)

train_transform = A.Compose([
    # Geometric transforms
    A.HorizontalFlip(p=0.5),  # Safe - images are L/R symmetric

    A.ShiftScaleRotate(
        shift_limit=0.05,      # 5% shift
        scale_limit=0.1,       # ±10% zoom
        rotate_limit=10,       # ±10° rotation (horizon varies naturally by 37%)
        border_mode=0,
        p=0.5
    ),

    # Color transforms (dataset has HIGH variation)
    A.RandomBrightnessContrast(
        brightness_limit=0.2,  # ±20% (dataset varies 136%)
        contrast_limit=0.2,
        p=0.5
    ),

    A.RGBShift(
        r_shift_limit=15,      # Color channels vary by 150+ values
        g_shift_limit=15,
        b_shift_limit=15,
        p=0.3
    ),

    # Blur (focus varies in real driving)
    A.GaussianBlur(blur_limit=(3, 5), p=0.2),

    # CoarseDropout (forces model to learn context, not just texture)
    A.CoarseDropout(
        max_holes=8,
        max_height=32,
        max_width=32,
        fill_value=0,
        p=0.3
    ),
])

def load_image_raw(path):
    img = Image.open(path).convert("RGB").resize(IMG_SIZE, Image.BILINEAR)
    return np.array(img, dtype=np.uint8)

def load_mask_raw(path):
    mask = np.array(Image.open(path).resize(IMG_SIZE, Image.NEAREST), dtype=np.uint16)
    class_mask = np.zeros(mask.shape, dtype=np.uint8)
    for value, idx in VALUE_TO_CLASS.items():
        class_mask[mask == value] = idx
    return class_mask

def load_image(path):
    img = Image.open(path).convert("RGB").resize(IMG_SIZE, Image.BILINEAR)
    return np.array(img, dtype=np.float32) / 255.0

class OffroadDataset(keras.utils.Sequence):
    def __init__(self, img_paths, mask_paths, augment=False):
        self.img_paths, self.mask_paths = list(img_paths), list(mask_paths)
        self.augment = augment

    def __len__(self): return len(self.img_paths) // BATCH_SIZE

    def __getitem__(self, idx):
        batch_imgs = self.img_paths[idx*BATCH_SIZE:(idx+1)*BATCH_SIZE]
        batch_masks = self.mask_paths[idx*BATCH_SIZE:(idx+1)*BATCH_SIZE]

        images, masks = [], []
        for img_path, mask_path in zip(batch_imgs, batch_masks):
            img = load_image_raw(img_path)
            mask = load_mask_raw(mask_path)

            if self.augment:
                augmented = train_transform(image=img, mask=mask)
                img, mask = augmented['image'], augmented['mask']

            images.append(img.astype(np.float32) / 255.0)
            masks.append(mask)

        return np.array(images), np.array(masks)

    def on_epoch_end(self):
        idx = np.random.permutation(len(self.img_paths))
        self.img_paths = [self.img_paths[i] for i in idx]
        self.mask_paths = [self.mask_paths[i] for i in idx]

# Load and split
img_paths = sorted(glob.glob(f"{TRAIN_IMAGES_DIR}/*.png"))
mask_paths = sorted(glob.glob(f"{TRAIN_MASKS_DIR}/*.png"))
train_imgs, val_imgs, train_masks, val_masks = train_test_split(img_paths, mask_paths, test_size=0.15, random_state=42)

train_ds = OffroadDataset(train_imgs, train_masks, augment=True)
val_ds = OffroadDataset(val_imgs, val_masks, augment=False)
print(f"Train: {len(train_imgs)}, Val: {len(val_imgs)}")
print("Augmentations: HFlip, ShiftScaleRotate(10°), Brightness(±20%), RGBShift(±15), Blur, Dropout")

Train: 2697, Val: 477
Augmentations: HFlip, ShiftScaleRotate(10°), Brightness(±20%), RGBShift(±15), Blur, Dropout


/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipython-input-1561171283.py:37: UserWarning: Argument(s) 'max_holes, max_height, max_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(


In [16]:
def dice_loss(y_true, y_pred):
    y_true = tf.one_hot(tf.cast(y_true, tf.int32), NUM_CLASSES)
    y_pred = tf.nn.softmax(y_pred, axis=-1)
    num = 2 * tf.reduce_sum(y_true * y_pred, axis=(1, 2))
    den = tf.reduce_sum(y_true + y_pred, axis=(1, 2))
    return 1 - tf.reduce_mean((num + 1) / (den + 1))

def combined_loss(y_true, y_pred):
    ce = keras.losses.SparseCategoricalCrossentropy(from_logits=True)(y_true, y_pred)
    return ce + dice_loss(y_true, y_pred)

def mean_iou(y_true, y_pred):
    y_pred = tf.argmax(y_pred, axis=-1)
    y_true, y_pred = tf.cast(y_true, tf.int64), tf.cast(y_pred, tf.int64)
    ious = []
    for i in range(NUM_CLASSES):
        inter = tf.reduce_sum(tf.cast((y_true==i) & (y_pred==i), tf.float32))
        union = tf.reduce_sum(tf.cast((y_true==i) | (y_pred==i), tf.float32))
        ious.append((inter + 1e-7) / (union + 1e-7))
    return tf.reduce_mean(ious)

In [17]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=combined_loss,
    metrics=[mean_iou]
)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        f"{CHECKPOINT_DIR}/best_model.keras",
        monitor="val_mean_iou", mode="max", save_best_only=True, verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        f"{CHECKPOINT_DIR}/latest_model.keras",
        save_best_only=False, verbose=0
    ),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
]

print(f"Training {NUM_CLASSES}-class model for {EPOCHS} epochs...")
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=callbacks)

Training 10-class model for 20 epochs...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
674/674 ━━━━━━━━━━━━━━━━━━━━ 0s 531ms/step - loss: 1.4099 - mean_iou: 0.3652
Epoch 1: val_mean_iou improved from -inf to 0.15778, saving model to /content/drive/MyDrive/TerraSeg_Hackathon/checkpoints/best_model.keras
674/674 ━━━━━━━━━━━━━━━━━━━━ 490s 584ms/step - loss: 1.4096 - mean_iou: 0.3653 - val_loss: 3.0710 - val_mean_iou: 0.1578 - learning_rate: 1.0000e-04
Epoch 2/20
674/674 ━━━━━━━━━━━━━━━━━━━━ 0s 532ms/step - loss: 1.0100 - mean_iou: 0.5251
Epoch 2: val_mean_iou improved from 0.15778 to 0.54378, saving model to /content/drive/MyDrive/TerraSeg_Hackathon/checkpoints/best_model.keras
674/674 ━━━━━━━━━━━━━━━━━━━━ 386s 573ms/step - loss: 1.0100 - mean_iou: 0.5251 - val_loss: 0.9332 - val_mean_iou: 0.5438 - learning_rate: 1.0000e-04
Epoch 3/20
674/674 ━━━━━━━━━━━━━━━━━━━━ 0s 532ms/step - loss: 0.8498 - mean_iou: 0.5547
Epoch 3: val_mean_iou improved from 0.54378 to 0.58571, saving model to /content/drive/MyDrive/TerraSeg_Hackathon/checkpoints/best_model.keras
674/674 ━━━━

In [21]:
# Load best model
model = keras.models.load_model(
    f"{CHECKPOINT_DIR}/best_model.keras",
    custom_objects={'combined_loss': combined_loss, 'mean_iou': mean_iou}
)
print("Best model loaded!")

def rle_encode(mask):
    pixels = np.concatenate([[0], mask.flatten(order="F"), [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(map(str, runs))

def predict_with_tta(model, img):
    """
    TTA: Horizontal flip only (safe based on analysis).
    No vertical flip - would harm accuracy.
    """
    img_batch = np.expand_dims(img, axis=0)

    # Original
    pred1 = model.predict(img_batch, verbose=0)[0]

    if USE_TTA:
        # Horizontal flip
        pred2 = model.predict(np.expand_dims(np.fliplr(img), axis=0), verbose=0)[0]
        pred2 = np.fliplr(pred2)

        pred_avg = (pred1 + pred2) / 2.0
    else:
        pred_avg = pred1

    return np.argmax(pred_avg, axis=-1)

print(f"TTA: {USE_TTA} (horizontal flip only)")

Best model loaded!
TTA: True (horizontal flip only)


In [22]:
# Run inference
test_paths = sorted(glob.glob(f"{TEST_IMAGES_DIR}/*.png"))
print(f"Processing {len(test_paths)} test images...")

rows = []
for path in tqdm(test_paths):
    img = load_image(path)
    pred = predict_with_tta(model, img)

    # Resize to 540x960
    pred = np.array(Image.fromarray(pred.astype(np.uint8)).resize((960, 540), Image.NEAREST))

    # Convert to original values then binary
    values = np.zeros(pred.shape, dtype=np.uint16)
    for idx, val in CLASS_TO_VALUE.items():
        values[pred == idx] = val
    binary = np.isin(values, list(FG_VALUES)).astype(np.uint8)

    rows.append({"image_id": os.path.splitext(os.path.basename(path))[0], "encoded_pixels": rle_encode(binary)})

# Save
df = pd.DataFrame(rows)
df.to_csv("submission.csv", index=False)
df.to_csv(f"{SUBMISSION_DIR}/submission.csv", index=False)
print(f"\n✅ Saved: {len(df)} entries")
df.head()

Processing 1002 test images...


100%|██████████| 1002/1002 [05:11<00:00,  3.22it/s]



✅ Saved: 1002 entries


,image_id,encoded_pixels
0,0001,1 21 117 1 158 5 210 48 263 28 318 51 381 4 40...
1,0002,1 22 25 48 140 9 175 2 235 18 258 2 267 93 364...
2,0003,1 84 94 6 111 4 133 15 170 11 248 4 256 7 266 ...
3,0004,1 83 174 4 251 28 285 152 439 45 515 5 525 99 ...
4,0005,1 80 169 9 247 40 293 151 450 10 477 9 500 8 5...


In [23]:
from google.colab import files
files.download("submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>